In [1]:
!pip install geopandas matplotlib contextily openeo


[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


## Define an area of interest

In [5]:
aoi = {
    'east': 22.136334940987638,  # max longitude
    'south': 65.63560585326866,  # min latitude
    'west': 21.79257719972466,   # min longitude
    'north': 65.78212032521256   # max latitude
}

## Plot an area of interest

In [7]:
import folium
import io
import numpy as np
import openeo
import json
import rasterio
from pathlib import Path
from config import *
from demo_helpers import show_result
from demo_regions import s2,s3
from config import user,passwd,eo_service_url
from demo_regions import s2
from matplotlib import pyplot as plt
from IPython.display import display, IFrame

center_lat = (aoi['north'] + aoi['south']) / 2
center_lon = (aoi['east'] + aoi['west']) / 2

m = folium.Map(location=[center_lat, center_lon], zoom_start=2, tiles="OpenStreetMap")

folium.Rectangle(
    bounds=[
        [aoi['south'], aoi['west']],
        [aoi['north'], aoi['east']]
    ],
    color="red",
    weight=2,
    fill=True,
    fill_opacity=0.2
).add_to(m)

## Save the map as an HTML file
#m.save("bounding_box_map.html")

# Display the map directly in the notebook
display(IFrame("bounding_box_map.html", width=700, height=500))

## OpenEO utility functions

In [24]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [20, 20]

connection = openeo.connect(eo_service_url)

print(eo_service_url)
print(user)
print(connection.authenticate_basic(username=user, password=passwd))

def poll_job(job, polling_interval=10):
    while True:
        status = job.status()
    
        print(f"Job status: {status}")
        
        if status == "finished":
            print("Job has finished.")
            break
            
        time.sleep(polling_interval)

def plot_file(file_path: str, title: str=""):
    with rasterio.open(file_path) as im:
        r = im.read(1)
        g = im.read(2)
        b = im.read(3)

        r_normalized = normalize(r)
        g_normalized = normalize(g)
        b_normalized = normalize(b)

        rgb = np.dstack((r_normalized, g_normalized, b_normalized))

    _, ax = plt.subplots(figsize=(40, 40))
    ax.imshow(rgb)
    ax.set_title(title)
    plt.show()

def convert_gtiff_to_img(file_path: str, output_png_path: str, title: str=""):
    # Open the GeoTIFF file
    with rasterio.open(file_path) as im:
        r = im.read(1)
        g = im.read(2)
        b = im.read(3)

        r_normalized = normalize(r)
        g_normalized = normalize(g)
        b_normalized = normalize(b)

        rgb = np.dstack((r_normalized, g_normalized, b_normalized))

    plt.imsave(output_png_path, rgb)

def normalize(array):
    array_min, array_max = np.percentile(array, (1, 99))
    array = np.clip(array, array_min, array_max)
    return ((array - array_min) / (array_max - array_min) * 255).astype(np.uint8)

class Dict2Class(object):
    def __init__(self, my_dict):

        for key in my_dict:
            if type(my_dict[key]) is dict:
                if key in ["bbox", "timespans", "polygons"]:
                    my_dict[key] = Dict2Class(my_dict[key])
            setattr(self, key, my_dict[key])

s2 = {
    "s2_msi_l2a": "s2_msi_l2a",
    "bbox": {
    }
}
s2 = Dict2Class(s2)

https://openeo.digitalearth.se
testuser
<Connection to 'https://openeo.digitalearth.se/' with BasicBearerAuth>


## Create a new job

In [25]:
cube = connection.load_collection(collection_id=s2.s2_msi_l2a,
                         spatial_extent=aoi,
                         temporal_extent=["2024-08-02T00:00:00Z", "2024-08-10T10:00:00Z"],
                         bands=['b04', 'b03', 'b02', 'b08'],
                        )

job = cube.create_job(out_format="gtiff",
    title= "Max time demo job1",
    description = "This job was created from a notebook")

print("Creating OpenEO job, ID=", job.job_id)

Creating OpenEO job, ID= 9b82a468-33d1-4e1c-873e-de666e69d914


## Start and wait

In [26]:
print("Starting OpenEO job, ID=", job.job_id)
print("Waiting ...")
job.start_and_wait()
poll_job(job)

results = job.get_results()
results.get_metadata()
results.download_files("data/out")

Starting OpenEO job, ID= 9b82a468-33d1-4e1c-873e-de666e69d914
Waiting ...
0:00:00 Job '9b82a468-33d1-4e1c-873e-de666e69d914': send 'start'
0:00:00 Job '9b82a468-33d1-4e1c-873e-de666e69d914': running (progress 0.0%)
0:00:05 Job '9b82a468-33d1-4e1c-873e-de666e69d914': running (progress 0.0%)
0:00:11 Job '9b82a468-33d1-4e1c-873e-de666e69d914': running (progress 0.0%)
0:00:19 Job '9b82a468-33d1-4e1c-873e-de666e69d914': running (progress 0.0%)
0:00:29 Job '9b82a468-33d1-4e1c-873e-de666e69d914': finished (progress 0.0%)
Job status: finished
Job has finished.


[PosixPath('data/out/process.json'),
 PosixPath('data/out/out_2024_08_05T10_06_01.tif'),
 PosixPath('data/out/out_2024_08_06T10_25_59.tif'),
 PosixPath('data/out/out_2024_08_08T10_20_21.tif'),
 PosixPath('data/out/out_2024_08_03T10_15_59.tif'),
 PosixPath('data/out/stac.json'),
 PosixPath('data/out/9b82a468-33d1-4e1c-873e-de666e69d914.txt'),
 PosixPath('data/out/py_proc_std_out.txt'),
 PosixPath('data/out/py_proc_std_err.txt'),
 PosixPath('data/out/job-results.json')]

## Cleanup

In [13]:
directory = Path('data/out')

print("Cleaning junk files ...")
for file in directory.iterdir():
    if file.is_file() and file.suffix != '.tif':
        file.unlink()
        print(f"Deleted: {file}")

print("Deletion complete.")

Cleaning junk files ...
Deleted: data/out/py_proc_std_err.txt
Deleted: data/out/stac.json
Deleted: data/out/process.json
Deleted: data/out/out_2024_08_05T10_06_01.jpg
Deleted: data/out/out_2024_08_03T10_15_59.jpg
Deleted: data/out/job-results.json
Deleted: data/out/out_2024_08_06T10_25_59.jpg
Deleted: data/out/py_proc_std_out.txt
Deleted: data/out/d704d241-91b3-4649-83dc-97dfa2194f8c.txt
Deleted: data/out/out_2024_08_08T10_20_21.jpg
Deletion complete.


## Convert to JPEG

In [14]:
# plot_file('data/out/out_2024_08_06T10_25_59.tif')
def convert_all_tif_to_jpg(directory: str):
    dir_path = Path(directory)
    
    for tif_file in dir_path.glob('*.tif'):
        jpg_file = tif_file.with_suffix('.jpg')
        convert_gtiff_to_img(str(tif_file), str(jpg_file))
        print(f"Converted {tif_file} to {jpg_file}")

print("Converting to GeoTiff to JPEG ...")

convert_all_tif_to_jpg('data/out')

Converting to GeoTiff to JPEG ...
Converted data/out/out_2024_08_06T10_25_59.tif to data/out/out_2024_08_06T10_25_59.jpg
Converted data/out/out_2024_08_08T10_20_21.tif to data/out/out_2024_08_08T10_20_21.jpg
Converted data/out/out_2024_08_03T10_15_59.tif to data/out/out_2024_08_03T10_15_59.jpg
Converted data/out/out_2024_08_05T10_06_01.tif to data/out/out_2024_08_05T10_06_01.jpg
